# Scape Sector information for one Mutual fund from Yahoo Finance

This works using splinter to open the url in the browser.  We must do it this way because the data we are looking for is not it the HTML if look at it outside of a browser. The data is loaded with the webpage by a java script.  So if we just did a "requests.get(url)" the data we are after would not be there.  We must 
load a browser and scrape/pars that.

### Next Steps in Python
* turn this into a function that can loop through a list of mutual funds
* Build a data frame that has a row for each of those mutual funds
* Convert the values in the data frame from strings to values
* Export that data frame as a CSV file

### Next step in Excel 
* Import CSV file in my financial spreadsheet
* Report my exposer to each of these sectors.


In [9]:
# Website:  https://finance.yahoo.com/quote/VTSAX/holdings?p=VTSAX

# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import json

In [5]:
# This function is used to splits a string into 2 parts at the first number.
# A dictionary is created with the first string as the key and the second string as the value
def split_string_at_first_number(string):
    index = len(string)
    for i, char in enumerate(string):
        if char.isdigit():
            index = i
            break
    key = string[:index]
    value = string[index:]
    dictionary = {key: value}
    return dictionary

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Mutual fund name
fund = 'VTSAX'

# Yahoo finance base URL
base_url = 'https://finance.yahoo.com/quote/VTSAX/holdings?p='

# full url that I want to scrap.
url = base_url + fund

# open a browser to the required URL
browser.visit(url)

In [4]:
# Create a Beautiful Soup object

# This will open the url in the browser.  We must do it this way because the data we are looking for is not it the HTML if look at it outside of a browser.
# The data is loaded with the webpage by a java script.  So if we just did a "requests.get(url)" the data we are after would not be there.  We must 
# load a browser and scrape/pars that.

html = browser.html
soup = soup(html, 'html.parser')

In [6]:
# get news story titles

Industry_sectors = []

# Start a dictionary with the key-value pair for the fund name.
Industry_sec ={'Fund_Code': fund}

# get the industry sector information from the url
# The search strings come from "inspecting" the website where the data is located.
# The website does not unquicly identify sector information. It get extra information that will need to be cleaned out 
Industry_sector = soup.find_all('div', class_='Bdbw(1px) Bdbc($seperatorColor) Bdbs(s) H(25px) Pt(10px)')

for sector in Industry_sector:
            # print('-------------')
            # print(sector.text)

            # return sector name and value
            sec = str(sector.text)

            # seperate the name and value and update the dictionary Industry_sec with the new key-value pair
            Industry_sec.update (split_string_at_first_number(sec))

            # biult a list of sector information - right now this is not used.
            Industry_sectors.append(sector.text)

In [7]:
# build a dataframe using the Industry_sec dictionary
df = pd.DataFrame(Industry_sec, index =[0])
df.head()

,Fund_Code,Cash,Stocks,Bonds,Others,Preferred,Convertable,Basic Materials,Consumer Cyclical,Financial Services,...,Credit QualityN/AN/A,US Government,AAAN/A,AAN/A,AN/A,BBBN/A,BBN/A,BN/A,Below BN/A,OthersN/A
0,VTSAX,0.95%,99.04%,0.00%,0.00%,0.00%,0.00%,2.69%,10.53%,13.93%,...,,0.00%,,,,,,,,


In [8]:
# This is a list of the columns that are actually industry sectors
industry_sector_col = [
      'Fund_Code', 
      'Basic Materials', 
      'Consumer Cyclical', 
      'Financial Services',
      'Real Estate', 
      'Consumer Defensive', 
      'Healthcare', 'Utilities',
      'Communication Services', 
      'Energy', 
      'Industrials', 
      'Technology']

# Make a data frame that only includes the industry sector key-value pairs.
df_industry = df[industry_sector_col]
df_industry.head()

,Fund_Code,Basic Materials,Consumer Cyclical,Financial Services,Real Estate,Consumer Defensive,Healthcare,Utilities,Communication Services,Energy,Industrials,Technology
0,VTSAX,2.69%,10.53%,13.93%,3.43%,6.47%,14.14%,2.68%,6.97%,4.68%,9.75%,23.77%
